# 초기 설정

- 필요패키지 import

In [285]:
import numpy as np
import pandas as pd
import time
import requests # 크롤링에 사용하는 패키지
from bs4 import BeautifulSoup # html 변환에 사용함
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

- `ad.csv`에서 content_id 불러옴

In [2]:
ad_df = pd.read_csv('data/ad.csv')

In [7]:
content_id = ad_df['content_id']

- 셀레니움 설정

In [87]:
driver = webdriver.Chrome('.//chromedriver.exe')

In [88]:
url = 'https://m.bunjang.co.kr/'
driver.get(url)

# content_id 기반으로 추가 정보 크롤링

- `b_pay` : 번개페이 여부  (번개페이 사용시 `1`, 아니면 `0`)
- `place` : 거래 지역
- `text` : 판매자가 등록한 상품 정보
- `delivery_fee` : 배송비 여부 (`배송비포함`, `배송비 별도`)
- `category` : 세부 카테고리 명(한글)
- `status` : 등록되었던 상품의 현 상태 : `판매중`, `거래 완료`, `삭제됨`

In [95]:
content_data = []

for cid in content_id:
    url = f'https://m.bunjang.co.kr/products/{cid}'
    driver.get(url)
    cnt += 1
    tt = 2 + np.random.rand(1)[0]  
    time.sleep(tt)
    # 번개페이 여부
    try : 
        b_pay = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[4]/div[1]/div/div[2]/div/div[2]/div/div[1]/div[1]/div[2]/div[2]/span/img').text
        b_pay = 1
    except:
        b_pay = 0
    # 거래 지역
    try:
        place = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[4]/div[1]/div/div[2]/div/div[2]/div/div[1]/div[2]/div[2]/div[4]/div[2]').text
    except:
        place = None
    # 상품 정보 : 판매자가 등록한 설명
    try:
        text = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[4]/div[1]/div/div[4]/div[1]/div[2]/div[1]').text
    except:
        text = None
    # 배송비
    try:
        delivery_fee = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[4]/div[1]/div/div[2]/div/div[2]/div/div[1]/div[2]/div[2]/div[3]/div[2]').text
    except:
        delivery_fee = None
    # 카테고리
    try:
        category = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[4]/div[1]/div/div[4]/div[1]/div[2]/div[1]/div[2]/div[3]/div[2]/div[2]/a/span').text
    except:
        category = None
    # 게시글의 현 상태    
    if driver.find_element_by_xpath('//*[@id="root"]/div/div/div[4]/div/div/div[2]/div[1]').text[:9] == '이미 삭제된 상품':
        status = '삭제됨'
    elif driver.find_element_by_xpath('//*[@id="root"]/div/div/div[4]/div/div/div[2]/div[1]').text[:10] == '이미 거래가 완료된':
        status = '거래 완료'
    else:
        status ='판매중'
    # 각 id별로 합쳐서 저장
    content_dict = {'id':cid, 'status': status, 'b_pay':b_pay, 'place':place, 'text':text, 'delivery_fee':delivery_fee,, 'category': category}
    content_data.append(content_dict)
# 데이터 프레임으로 만든 후 저장
df = pd.DataFrame(content_data)
df.to_csv('ad_content_data.csv', index=False)

In [50]:
dff[dff['status'] == '판매중']

,id,status,b_pay,place,text,delivery_fee
0,157826057,판매중,1,서울특별시 용산구 한강로동,"상품정보\n[ 컴퓨터 상세 사양 ] \n(SSD, 케이스만 신품, 나머지 중고)\n...",배송비포함
3,152625380,판매중,1,서울특별시 강남구 논현2동,"상품정보\n★ 번개장터 구매고객 특별이벤트 ★\n\n1,구입후 중고폰 100일 제품...",배송비포함
6,163127295,판매중,1,충청북도 청주시 청원구 율량.사천동\n지역인증,상품정보\n상품명:러블리트렌치 코트 \n\n봄에 엄청난 인기상품이였던 소재부터\n요...,배송비 별도
7,137118036,판매중,1,서울특별시 강남구 삼성동,"상품정보\n★ 번개장터 구매고객 특별이벤트 ★\n\n1,구입후 중고폰 100일 제품...",배송비포함
8,162362067,판매중,1,서울특별시 서초구 서초1동\n지역인증,상품정보\n무려 8마리의 용과 화염 당초문으로 장식되어 있습니다.\n여의주를 쥔 가...,배송비 별도
...,...,...,...,...,...,...
1412,155571548,판매중,1,서울특별시 강남구 논현2동,상품정보\n고객님 안녕하세요!\n\nAIR OK 입니다^^\n\n저희 중고폰/액정파...,배송비포함
1414,162789993,판매중,1,전국,"상품정보\n검색창에서 "" 애슐리상점 "" \n\n잇미샤 루즈한라인 리본 블라우스 55...",배송비 별도
1418,84088254,판매중,1,부산광역시 남구 용호제1동\n지역인증,상품정보\n가장빠른 채팅\n\n카 카 오 톡 : bbb4883 \n\n카 카 오 톡...,배송비포함
1419,152049275,판매중,1,전국,상품정보\n품명:스위스밀리터리 에어슬림 무선 멀티 청소기 SMA-BV2200 \n시...,배송비포함


In [51]:
dff[dff['status'] == '거래 완료']

,id,status,b_pay,place,text,delivery_fee
1,162431268,거래 완료,0,NaN,NaN,NaN
18,162909690,거래 완료,0,NaN,NaN,NaN
28,162609608,거래 완료,0,NaN,NaN,NaN
32,162854569,거래 완료,0,NaN,NaN,NaN
35,163068537,거래 완료,0,NaN,NaN,NaN
...,...,...,...,...,...,...
1411,155602263,거래 완료,0,NaN,NaN,NaN
1413,160744536,거래 완료,0,NaN,NaN,NaN
1415,162671672,거래 완료,0,NaN,NaN,NaN
1417,162431076,거래 완료,0,NaN,NaN,NaN


In [52]:
dff[dff['status'] == '삭제됨']

,id,status,b_pay,place,text,delivery_fee
2,162806215,삭제됨,0,NaN,NaN,NaN
4,161572473,삭제됨,0,NaN,NaN,NaN
5,159624182,삭제됨,0,NaN,NaN,NaN
11,163063044,삭제됨,0,NaN,NaN,NaN
12,151970858,삭제됨,0,NaN,NaN,NaN
...,...,...,...,...,...,...
1420,160809748,삭제됨,0,NaN,NaN,NaN
1421,159900430,삭제됨,0,NaN,NaN,NaN
1424,163149174,삭제됨,0,NaN,NaN,NaN
1425,160955756,삭제됨,0,NaN,NaN,NaN


# 거래 완료된 데이터 추가 크롤링

- 페이지 이동 `.click()` 추가해서 거래 완료된 데이터도 크롤링하도록 함

In [136]:
content_complete = []

for cid in df[df['status'] == '거래 완료']['id']:
    url = f'https://m.bunjang.co.kr/products/{cid}'
    driver.get(url)
    cnt += 1
    tt = 3 + np.random.rand(1)[0]  
    time.sleep(tt)
    # 페이지 이동
    driver.find_element_by_xpath('//*[@id="root"]/div/div/div[4]/div[1]/div/div[2]/div[1]/a/div[2]/div[1]').click()
    # 번개페이 여부
    try : 
        b_pay = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[4]/div[1]/div/div[2]/div/div[2]/div/div[1]/div[1]/div[2]/div[2]/span/img').text
        b_pay = 1
    except:
        b_pay = 0
    # 거래 지역
    try:
        place = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[4]/div[1]/div/div[2]/div/div[2]/div/div[1]/div[2]/div[2]/div[4]/div[2]').text
    except:
        place = None
    # 상품 정보 : 판매자가 등록한 설명
    try:
        text = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[4]/div[1]/div/div[4]/div[1]/div[2]/div[1]').text
    except:
        text = None
    # 배송비
    try:
        delivery_fee = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[4]/div[1]/div/div[2]/div/div[2]/div/div[1]/div[2]/div[2]/div[3]/div[2]').text
    except:
        delivery_fee = None
    # 카테고리
    try:
        category = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[4]/div[1]/div/div[4]/div[1]/div[2]/div[1]/div[2]/div[3]/div[2]/div[2]/a/span').text
    except:
        category = None
    # 게시글의 현 상태    
    if driver.find_element_by_xpath('//*[@id="root"]/div/div/div[4]/div/div/div[2]/div[1]').text[:9] == '이미 삭제된 상품':
        status = '삭제됨'
    elif driver.find_element_by_xpath('//*[@id="root"]/div/div/div[4]/div/div/div[2]/div[1]').text[:10] == '이미 거래가 완료된':
        status = '거래 완료'
    else:
        status ='판매중'
    # 각 id별로 합쳐서 저장
    content_dict = {'id':cid, 'status': status, 'b_pay':b_pay, 'place':place, 'text':text, 'delivery_fee':delivery_fee, 'category': category}
    content_complete.append(content_dict)
# 데이터 프레임으로 만든 후 저장
df_complete = pd.DataFrame(content_complete)
df_complete.to_csv('ad_content_data_complete.csv', index=False)

# 크롤링한 데이터들 병합

In [141]:
ad_c = pd.read_csv('data/ad_content_data.csv')
ad_c_new = pd.read_csv('data/ad_content_data_new.csv')
ad_c_comp = pd.read_csv('data/ad_content_data_complete.csv')

- 거래 완료된 따로 크롤링 후 병합

In [202]:
# 거래완료인 데이터들 drop
complete_ind = ad_c[ad_c['status']=='거래 완료'].index.tolist()
ad_drop = ad_c.drop(complete_ind)
# 거래완료된 상품들에 대해 새로 크롤링한 데이터 추가
ad_new = ad_drop.append(ad_c_comp)
ad_new = ad_new.reset_index().drop('index', axis=1)

In [213]:
ad_new

,id,status,b_pay,place,text,delivery_fee,category
0,157826057,판매중,1,서울특별시 용산구 한강로동,"상품정보\n[ 컴퓨터 상세 사양 ] \n(SSD, 케이스만 신품, 나머지 중고)\n...",배송비포함,데스크탑
2,162806215,삭제됨,0,NaN,NaN,NaN,None
3,152625380,판매중,1,서울특별시 강남구 논현2동,"상품정보\n★ 번개장터 구매고객 특별이벤트 ★\n\n1,구입후 중고폰 100일 제품...",배송비포함,스마트폰
4,161572473,삭제됨,0,NaN,NaN,NaN,None
5,159624182,삭제됨,0,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...
289,155602263,판매중,1,경기도 김포시 사우동,상품정보\n▲LG G7 (A급) 64GB 블루\n\n외관은 생활기스 조금 있으며 ...,배송비 별도,스마트폰
290,160744536,판매중,1,서울특별시 용산구 한강로동,"상품정보\nQHD 포토샵, 영상편집OK, 최상의 게이밍 성능!!\n\n✨ 신품(정품...",배송비포함,데스크탑
291,162671672,판매중,0,서울특별시 관악구 청룡동,상품정보\n가슴 50\n기장 62\n바지기장 90\n실크100%\n물세탁 가능한 실...,배송비 별도,언더웨어/홈웨어
292,162431076,판매중,0,경기도 평택시 신장1동,상품정보\n♥ 핸드폰장인만의 혜택 ♥\n\n1.전상품 필름 + 케이스 + 고속충전케...,배송비포함,스마트폰


- 크롤링 도중에 거래완료 -> 삭제 된 데이터 하나 수동으로 처리

In [277]:
ad_new = ad_new.append({'id':161397461, 'status': '삭제됨', 'b_pay':0, 'place':None, 'text':None, 'delivery_fee':None, 'category': None}, ignore_index=True)

### 결측치 확인

결측치 2개 확인

In [235]:
ad_new[ad_new['status']!='삭제됨']['category'].isnull()

2

결측치 인덱스 찾기

In [238]:
## 결측치 2개 확인
ad_non['category'][ad_non['category'].isna()]

561    NaN
883    NaN
Name: category, dtype: object

- 수동으로 결측 채워주기 (크롤링 과정에서 누락된 듯)

In [239]:
ad_non.loc[561]

id                         133347162
status                           판매중
b_pay                              1
place                             전국
text            13fw 릭오웬스 디트로이트컷 코팅진
delivery_fee                  배송비 별도
category                         NaN
Name: 561, dtype: object

In [271]:
ad_new.loc[561, 'category'] = '이어폰/헤드폰'

In [240]:
ad_non.loc[883]

id                                                      163101393
status                                                        판매중
b_pay                                                           0
place                                                경기도 파주시 운정3동
text            상품정보\n개봉만 한 진열 상품 입니다\n거래지역\n경기도 파주시 운정3동\n카테고...
delivery_fee                                               배송비 별도
category                                                      NaN
Name: 883, dtype: object

In [268]:
ad_new.loc[883, 'category'] = '유모차'

### 확인

결측치의 개수와 삭제됨의 개수가 일치함을 확인

In [282]:
ad_new.category.isnull().sum()

484

데이터가 없어서 `None`를 넣어 준 데이터 말고는 결측이 없는 것 확인

In [284]:
len(ad_new[ad_new['status']=='삭제됨'])

484

# 데이터 저장

In [276]:
ad_new.to_csv('data/ad_new.csv', index=False)